In [1]:
import geopandas as gpd
from sqlalchemy import create_engine
import psycopg2

# import libraries
import geopandas as gpd
import pandas as pd
import numpy as np

# for interactive plots
import plotly.express as px
import seaborn
import pysal
import contextily as ctx

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from pointpats import centrography

import imageio
from IPython.display import Image

import matplotlib as mpl
# set font
mpl.rc('font',family='Hiragino Maru Gothic Pro')
# mpl.rc('font',family='YuMincho')

# library additionas for legends
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.patheffects as pe

pd.set_option("display.max_columns", None)

import plotly.io as pio

from adjustText import adjust_text

/Users/yohkawano/opt/anaconda3/envs/geos/lib/python3.11/site-packages/pysal/explore/segregation/network/network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
/Users/yohkawano/opt/anaconda3/envs/geos/lib/python3.11/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


## Import GIS layers from Postgresql

In [2]:
# Replace placeholders with your database credentials
DB_USER = ""
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "xymax"

# Construct the connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create the SQLAlchemy engine
con = create_engine(connection_string)


In [3]:
# import buildings
table_name = "SELECT * FROM H3building"
h3 = gpd.read_postgis(table_name, con)

In [5]:
# import tokyo cities
regions = gpd.read_file('../data/master/project_area_census_by_ku.geojson')

In [6]:
# set the crs to match
regions = regions.to_crs(h3.crs)

In [8]:
# put the ku's in a list
kus = regions['SIKUCHOSON'].unique().tolist()

In [9]:
# Perform spatial join to assign city to buildings
result = gpd.sjoin(h3, regions, how='left', op='intersects')

# Create a new "city" column in the result GeoDataFrame
result['city'] = result['SIKUCHOSON']

/Users/yohkawano/opt/anaconda3/envs/geos/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [10]:
result.to_file('h3.geojson', driver='GeoJSON')


In [ ]:
def building_by_ku(ku='墨田区'):
    
    number_of_buildings = len(result[result['SIKUCHOSON']==ku])

    if number_of_buildings > 0:
        fig, ax = plt.subplots(figsize=(8,8))
        
        subtitle_text = str(number_of_buildings)+' buildings'

        # Title text and font sizes
        line1_text = ku
        line2_text = str(len(result[result['SIKUCHOSON']==ku]))+' buildings'
        line1_fontsize = 50
        line2_fontsize = 12

        # Define outline color and width
        outline_color = 'white'
        outline_width = 2

        # Add title lines with outline color within the chart area
        title1 = ax.text(0.5, 0.7, line1_text, fontsize=line1_fontsize, ha='center', va='center', transform=ax.transAxes,
                        path_effects=[withStroke(linewidth=outline_width, foreground=outline_color)])
        title2 = ax.text(0.5, 0.61, line2_text, fontsize=line2_fontsize, ha='center', va='center', transform=ax.transAxes,
                        path_effects=[withStroke(linewidth=outline_width, foreground=outline_color)])

        result[result['SIKUCHOSON']==ku].plot(ax=ax,facecolor='none', edgecolor='black',alpha=0.8,linewidth=0.1)
        ax.axis('off')
       
        # Save the plot as an image file (e.g., PNG, JPEG, PDF, SVG)
        output_file = ku+'.png'  
        plt.savefig(output_file, dpi=300, bbox_inches='tight')  # You can adjust the DPI and other settings

building_by_ku()

In [ ]:
for ku in kus:
    building_by_ku(ku)